# **FIAP Pós-Tech - IA para Devs**

## Grupo 14 (1IADT) - Alunos

* Marcelo Henriques da Fonseca - RM353865
* Marcos Lopes da Silva Junior - RM353763
* Ricardo Báfica Pontes - RM353866

# **O que é Fine-Tuning?**

Fine-tuning é uma técnica poderosa que podemos utilizar para treinar modelos de Inteligência Artificial para propósitos específicos.

Ele envolve selecionar e ajustar um modelo pré-treinado (como GPT, Llama, BERT e outros) com um conjunto de dados específico para melhorar seu desempenho em alguma tarefa.

Neste projeto, vamos utilizar os dados de produtos da Amazon (tratados e divididos no Notebook Parte 1) para realizar o fine-tuning em um modelo GPT a fim de "ensina-lo" a falar sobre essas mercadorias.

# **Projeto**

A partir daqui, vamos utilizar as partes que dividimos do JSON e fizemos o upload para o Google Drive para fazer o fine-tuning de um modelo GPT-4o mini.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# instalando a biblioteca do OpenAI
!pip --quiet install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [3]:
# importando as bibliotecas necessárias
from openai import OpenAI
from google.colab import userdata
import textwrap

Abaixo usamos o "userdata.get" para recuperar a chave de API da OpenAI que foi definida nos "Secrets" no próprio Google Colab.

In [4]:
# criando o client, com a chave de acesso para a API
client = OpenAI(
  api_key=userdata.get('OPENAI_API_KEY')
)

Neste bloco, definiremos uma série de perguntas que serão respondidas pelo modelo base GPT-4o mini, a fim de termos uma base de comparação com o modelo fine-tuned.

In [60]:
# Definir as mensagens para enviar
messages = [
    "What Buffalo News said about 'The Birds of America: The Bien Chromolithographic Edition'?"
]

In [72]:
# acesso inicial, sem fine-tunig, para comparação futura

# Iterar sobre as mensagens e obter as respostas
for i, message_content in enumerate(messages):
    completion = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that informs customers about the contents of a store's products."},
            {"role": "user", "content": message_content}
        ],
        temperature=0.2
    )

    response = completion.choices[0].message.content
    print(f"Response {i+1}:\n{textwrap.fill(response, 80)}\n")

Response 1:
I don't have access to specific articles or content from the Buffalo News or any
other publication. However, "The Birds of America: The Bien Chromolithographic
Edition" is a well-known work that features stunning illustrations of birds by
John James Audubon. The Bien edition, published in the 19th century, is notable
for its high-quality chromolithographic reproductions of Audubon's original
paintings. If you're looking for a review or specific commentary from the
Buffalo News, I recommend checking their archives or website for the most
accurate and detailed information.



Agora iremos converter o nosso arquivo JSON do Google Drive para um arquivo JSONL que é necessário para realizar o fine-tuning pela OpenAI. Também ajustaremos o conteúdo do JSON para um formato compatível para o treinamento.

Ao final, salvaremos o novo arquivo de novo no Google Drive.

In [ ]:
import json

# Caminho do arquivo JSON original
arquivo_json = '/content/drive/MyDrive/FIAP/IA_para_Devs/Tech_Challenge_3/trn_100k_1.json'
# Caminho para o arquivo JSONL que será gerado
arquivo_jsonl = '/content/drive/MyDrive/FIAP/IA_para_Devs/Tech_Challenge_3/trn_100k_1.jsonl'

dados = []

# Carregar o arquivo JSON
with open(arquivo_json, 'r') as f:
    for line in f:
        dados.append(json.loads(line))

# Verificar se os dados estão em formato de lista
if isinstance(dados, list):
    # Abrir o arquivo JSONL para escrita
    with open(arquivo_jsonl, 'w') as outfile:
        for item in dados:
            # Ajustar para o formato compatível com a OpenAI
            messages = {
                "messages": [
                    {"role": "system", "content": "You are a helpful assistant that informs customers about the contents of a store's products."},
                    {"role": "user", "content": item['title']},
                    {"role": "assistant", "content": item['content']}
                ]
            }
            json.dump(messages, outfile)
            outfile.write('\n')

print("Arquivo JSON convertido para JSONL com sucesso!")

Arquivo JSON convertido para JSONL com sucesso!


Nesse bloco iremos "subir" o nosso arquivo de treinamento para o ambiente OpenAI.

In [ ]:
client.files.create(
  file=open("/content/drive/MyDrive/FIAP/IA_para_Devs/Tech_Challenge_3/trn_100k_1.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-aH7jLrHN6m2i1rNII65wx59t', bytes=136677541, created_at=1726346872, filename='trn_100k_1.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

Para este projeto de Fine-Tuning decidimos utilizar o modelo GPT-4o mini da OpenAI. Ele muito bom e econômico, que se beneficia da arquitetura do GPT-4 em um modelo mais compacto.

Sendo uma versão mais leve, ele requer menos recursos computacionais e seu treinamento é mais rápido e barato, sendo uma ótimo opção para os propósitos didáticos deste projeto.

A OpenAI permite que selecionemos alguns hiperparâmetros para ajustar o nosso fine-tuning.

**Epoch:** refere-se a quantidade de vezes que o modelo processa o conjunto de dados, quantas vezes ele "passa" pelas informações de treinamento.

Um número maior de epochs pode fazer com que o modelo aprenda melhor dados, mas um número muito grande pode causar overfitting (fazendo com que ele apenas repita os dados que foram passados). Pelo contrário, um número insuficiente de epochs pode fazer com que o modelo não aprenda tão bem.

**Learning Rate Multiplier:** refere-se a taxa de aprendizado, que define a rapidez com que o modelo ajusta seus pesos durante o processo de treinamento.

Um Learning Rate Multiplier mais alto pode acelerar o treinamento, mas também pode fazer com que o modelo não converja em soluções ótimos, o tornando mais instável. Pelo contrário, um Learning Rate Multiplier mais baixo pode ajudar o modelo a fazer ajustes finos, mas também pode tornar o treinamento muito lento e fazer com que ele fique preso em mínimos locais, prejudicando a capacidade de aprendizado.

**Batch Size:** refere-se ao tamanho do lote de dados processado antes do modelo atualizar seus pesos.

Um Batch Size maior pode ser mais estável e resultar em um convergência mais suave, mas requer mais memória e maior tempo de treinamento. Pelo contrário, um Batch Size menor pode fazer com que o treinamento seja mais rápido, mas pode resultar em um modelo mais instável.

Após diversos testes com outros conjutos de dados menores, chegamos nos hiperparâmetros abaixo para o nosso fine-tuning final.

Um número de epochs de 2 pode parecer baixo, mas se mostrou adequado julgando os propósitos acadêmicos desse projeto e o tamanho dos dados a serem treinados. Decidimos evitar um número muito grande de epochs para reduzir o tempo de fine-tuning do modelo.

Neste caso, um learning rate multiplier um pouco mais alto de 1.8, nos ajudou a chegar em um bom modelo com um número de epochs reduzido.

O batch size escolhido de 66 é moderado, e não vimos necessidade de altera-lo neste caso.

In [ ]:
n_epochs = 2
learning_rate_multiplier = 1.8
batch_size = 66

parametros={"n_epochs": n_epochs, "learning_rate_multiplier": learning_rate_multiplier, "batch_size": batch_size}

O seguinte bloco cria um job de fine-tuning usando o modelo indicado (GPT-4o mini) e o nosso arquivo de treinamento.

In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-aH7jLrHN6m2i1rNII65wx59t",
  model="gpt-4o-mini-2024-07-18",
  ### PARÂMETROS
  hyperparameters=parametros
)

FineTuningJob(id='ftjob-4uaOwnmSMwdYaTdeaD1XsYvF', created_at=1726346895, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-JVwSQhJq81swAguZeRqpHGfb', result_files=[], seed=1288320048, status='validating_files', trained_tokens=None, training_file='file-aH7jLrHN6m2i1rNII65wx59t', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

Os 3 blocos abaixo recuperam algumas informações referentes ao nosso job de fine-tuning.

In [ ]:
# Lista 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-4uaOwnmSMwdYaTdeaD1XsYvF', created_at=1726346895, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-JVwSQhJq81swAguZeRqpHGfb', result_files=[], seed=1288320048, status='validating_files', trained_tokens=None, training_file='file-aH7jLrHN6m2i1rNII65wx59t', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None), FineTuningJob(id='ftjob-Ksoe0QtExNhQQpgl6UTBpkZG', created_at=1726342333, error=Error(code='invalid_training_file', message='The job failed due to an invalid training file. Invalid file format. Input file file-qi5PJGB680xpxOvNPXxPW9k2 is in the prompt-completion format, but the specified model gpt-4o-mini-2024-07-18 is a chat model and requires chat-formatted data. See 

In [ ]:
# Recupera o estado de um fine tune
client.fine_tuning.jobs.retrieve("ftjob-4uaOwnmSMwdYaTdeaD1XsYvF")

FineTuningJob(id='ftjob-4uaOwnmSMwdYaTdeaD1XsYvF', created_at=1726346895, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=1, batch_size=66, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-JVwSQhJq81swAguZeRqpHGfb', result_files=[], seed=1288320048, status='validating_files', trained_tokens=None, training_file='file-aH7jLrHN6m2i1rNII65wx59t', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
# Lista 10 eventos de um fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-4uaOwnmSMwdYaTdeaD1XsYvF", limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-7GCUzmF524kAQizhqADoGs9A', created_at=1726346895, level='info', message='Validating training file: file-aH7jLrHN6m2i1rNII65wx59t', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-mHYXgLM55i3idIB3rNqeqfsQ', created_at=1726346895, level='info', message='Created fine-tuning job: ftjob-4uaOwnmSMwdYaTdeaD1XsYvF', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

Com o modelo fine-tuned, vamos realizar um teste passando algumas perguntas e vendo como ele as responde.

In [75]:
# Iterar sobre as mensagens e obter as respostas
for i, message_content in enumerate(messages):
    completion_test = client.chat.completions.create(
        model="ft:gpt-4o-mini-2024-07-18:personal::A7XET4xh", # nosso modelo fine-tuned
        messages=[
            {"role": "system", "content": "You are a helpful assistant that informs customers about the contents of a store's products."},
            {"role": "user", "content": message_content}
        ],
        temperature=0.2
    )

    response_test = completion_test.choices[0].message.content
    print(f"Response {i+1}:\n{textwrap.fill(response, 80)}\n")
    print(f"Response Fine-Tuned {i+1}:\n{textwrap.fill(response_test, 80)}\n")

Response 1:
I don't have access to specific articles or content from the Buffalo News or any
other publication. However, "The Birds of America: The Bien Chromolithographic
Edition" is a well-known work that features stunning illustrations of birds by
John James Audubon. The Bien edition, published in the 19th century, is notable
for its high-quality chromolithographic reproductions of Audubon's original
paintings. If you're looking for a review or specific commentary from the
Buffalo News, I recommend checking their archives or website for the most
accurate and detailed information.

Response Fine-Tuned 1:
“An extraordinary work of art.” (Buffalo News)



Content no JSON: “Staggeringly beautiful.” (Buffalo News)

Comparando as duas respostas, podemos verificar que o modelo fine-tuned aprendeu com o conteúdo do JSON. Apesar da resposta não ser exata, o modelo tem conhecimento de que o Buffalo News fez um comentário positivo a respeito do livro, diferente do modelo base que não tem conhecimento algum sobre a citação do jornal.

# **Possível aprimoramento do modelo**

Para propósitos didáticos e levando em conta os requisitos do projeto, julgamos que um fine-tuning com 100.000 linhas é o suficiente para atingir nosso objetivo.

Entretanto, vale ressaltar que seria possível melhorar esse modelo com mais dados e expandir seu leque de conhecimento além das 100.000 que ele aprendeu.

Os códigos abaixo são um exemplo de como poderiamos realizar outros fine-tunings em cima do modelo fine-tuned para criar um modelo aprimorado.

Se quiséssemos fazer fine-tuning em um modelo para que ele aprendesse todos os mais de 1 milhão de dados do dataset original da Amazon, precisaríamos apenas treinar o modelo várias vezes, sempre iterando sobre o modelo fine-tuned mais recente com um arquivo com mais dados (trn_100k_2, trn_100k_3, trn_100k_4 ...).

In [ ]:
client.files.create(
  file=open("/content/drive/MyDrive/FIAP/IA_para_Devs/Tech_Challenge_3/trn_100k_2.jsonl", "rb"),
  purpose="fine-tune"
)

In [ ]:
# dispara job de criação do modelo, com fine-tuning, complementar aos treinamentos anteriores

# nome do modelo
# aqui podemos passar o nome de um modelo já ajustado anteriormente com fine-tuning
# isso iniciará um novo job de fine-tuning usando o modelo anterior como ponto de partida.
model_name = "ft:gpt-4o-mini-2024-07-18:personal::A7XET4xh"
# hiperparâmetros a serem utilizados
# a documentação da OpenAI recomenda treinar inicialmente sem especificar nenhum deles,
# permitindo a eles escolher um padrão com base no tamanho do conjunto de dados e, em seguida,
# ajustar se observarmos o seguinte:
# se o modelo não seguir os dados de treinamento tanto quanto esperado, aumente o número de épocas em 1 ou 2
#     isso é mais comum para tarefas para as quais existe uma única conclusão ideal (ou um pequeno conjunto de conclusões ideais semelhantes).
#     alguns exemplos incluem classificação, extração de entidade ou análise estruturada.
#     freqüentemente, essas são tarefas para as quais você pode calcular uma métrica de precisão final em relação a uma resposta de referência.
# se o modelo se tornar menos diversificado do que o esperado, diminua o número de épocas em 1 ou 2
#     isso é mais comum para tarefas para as quais há uma ampla gama de possíveis boas conclusões
n_epochs = 2

# se o modelo não parecer convergir, aumente o multiplicador da taxa de aprendizagem
learning_rate_multiplier = 1.8

batch_size = 66

# incluir na funcção abaixo, sfoc
hyperparameters={"n_epochs": n_epochs, "learning_rate_multiplier": learning_rate_multiplier, "batch_size": batch_size}

###########################################################################
# código abaixo comentado para não disparar nova criação inadvertidamente #
###########################################################################

# # treinamento do modelo
# client.fine_tuning.jobs.create(
#     training_file=training_file_id,
#     model=model_name,
#     hyperparameters=hyperparameters
# )

# **Conclusão**

Neste projeto criamos um modelo GPT-4o mini fine-tuned com dados de produtos da Amazon. Pudemos verificar as nuances no treinamento e observar as diversas de um modelo "base" para um modelo "especializado".

Na prática, o fine-tuning de um modelo de IA é muito útil. Treinar um modelo do zero é extremamente demorado, difícil e consome muitos recursos. O fine-tuning aproveita o conhecimento de um modelo já pré-treinado e o especializa.

Ele é muito técnica muito poderosa quando queremos personalizar um modelo para realizar tarefas específicas.

Casos de uso podem incluir o uso de fine-tuning para a criação de chatbots, assistentes virtuais, geração de conteúdos, análise de sentimentos, análises médicas e outros.